# 画像認識3：Convolutional Neural Network (CNN)
深層学習による画像認識のためのアーキテクチャであるCNNの実装を動かしてみましょう。   

最初のCNNアーキテクチャであるAlexNetの論文は以下で読むことができます（が、Webにわかりやすい解説が多数あります）。   
[Krizhevsky A, Sutskever I, Hinton GE. ImageNet classification with deep convolutional neural networks. NeurIPS. 2012. DOI: 10.1145/3065386](http://www.cs.toronto.edu/~hinton/absps/imagenet.pdf)   
今回は学習データが小規模なので、少しシンプルにしたアーキテクチャを使います。

## 今回挑戦する画像認識タスク
小規模な画像認識タスクとして、[CIFAR-10 datasets](https://www.cs.toronto.edu/~kriz/cifar.html)を使って、10クラスの物体認識を行います。   
CIFAR-10には、'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'の10クラスの物体について、それぞれ6000枚ずつ画像が用意されています。   
そのうち、各クラス5000枚（計50,000枚）を訓練データとし、1000枚（計10,000枚）を評価データとします。   
各画像は32x32 pixelと非常に小さい画像なので、条件を整えても正解率は80％弱(今回のデフォルトセッティングだと60%程度）とあまり高くありませんが、その分、短時間で学習が終わるので、深層学習の勉強用によく使われます。

## 利用するライブラリ
本プログラムでは、ニューラルネットワーク実装のためのライブラリとして、Kerasを利用しています。   

+ Keras Documentation: https://keras.io/ja/

また、本コードは以下のサイトのものを参考にしました。   

+ [Keras Documentation: Train a simple deep CNN on the CIFAR10 small images dataset.](https://keras.io/examples/cifar10_cnn/)

## 実行環境：Colaboratoryを強く推奨します

CPUでも実行できると思いますが、モデルの学習に計算負荷がかかります。   
Colaboratoryで「ランタイム＞ランタイムのタイプを変更」で「ハードウェアアクセラレータ」をGPUにしてから実行することをお勧めします。

# 1. ライブラリのインストール

本プログラムでは、ニューラルネットワーク実装のためのライブラリとして、Kerasを利用しています。   
'keras'と、そのバックエンドである'tensorflow'をインストールします。   

**tensorflowのインストールによる不具合が見られるようです。**
**Colaboratoryの使用を強くお勧めします。**

## 1.1 ローカルPCの場合（非推奨）

**必ず仮想環境を作ってからパッケージをインストールしてください。**   
ガイダンスの環境設定の資料を参照して、ライブラリのインストールをお願いします。
1. Anaconda Navigatorを開く
2. 「Environments」のタブを開き、中央のフレームで「base(root)」とある右側の「▶」をクリックし、"Open Terminal"をクリック
3. コマンドプロンプトから以下の二つのコマンドを実行  

``conda install -c anaconda tensorflow``   
``conda install -c anaconda keras``

**トラブルが起きる場合はColaboratoryをご利用ください。**

## 1.2 Colaboratoryの場合（推奨）
以下のセルを実行してください。   
**このセルはColaboratoryを起動するたびに必要となります**   

In [ ]:
##################################
### Colaboratoryのみ以下を実行 ###
##################################
import sys
if 'google.colab' in sys.modules:
    !pip install tensorflow
    !pip install keras

## ライブラリを読み込み

Kerasから今回使うライブラリをインポートしましょう。

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np

# 計算するたびに違う答えにならないよう、ランダムシードを設定する
np.random.seed(seed=0)

# 2. 学習データの準備

## CIFAR-10データセットをダウンロード
CIFAR-10 datasetsは様々な場面で使われているので、Kerasのライブラリの中に、データセットをダウンロードするための関数が用意されています。   
これを使ってデータセットをダウンロードし、読み込みましょう。   
データサイズは163MBです。自分のPCを使う場合は、ディスクの空き容量を確認してください。  
PCで以下のセルを実行すると、ダウンロードされたファイルがホームディレクトリの下の'.keras/datasets'の下に保存されています。   
次回から同じPC＋アカウントでこのコマンドを実行する際は、改めてダウンロードされることはありません。   
（Colaboratoryはランタイムを終了するとファイルがすべて消去されるため、毎回ダウンロードすることになります）


In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

入力画像を確認してみましょう。

x_trainは画像の集合で、 (50,000枚) x (32 pixel) x (32 pixel) x (RGB 3 channel)のテンソルです。
1枚目の画像サイズを見ると、32x32x3であることがわかります。
縦横32ピクセルの正方形の写真（とても小さい！）で、RGB3枚からなるカラー画像です。

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

また、y_trainには50,000枚の各画像の正解のラベルが入っています。   
このベクトルは縦ベクトル（列ベクトルともいう。横ではなく縦に値が連なるベクトル）であることに注意してください。

In [ ]:
print(len(y_train)) # 50,000枚の画像のラベルが入っている
print(y_train[:10]) # 最初の10枚のラベル（整数値で記録されている）
# ラベルの整数値とクラス名の関係を'labels'に代入
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print([labels[y_train[x][0]] for x in range(10)]) # 最初の10枚のラベル（テキストに変換したもの）

訓練画像を1枚表示させてみましょう。非常に画素が粗い画像であることがわかります。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 画像をいろいろと入れ替えてみましょう。
id =7
plt.imshow(x_train[id])
plt.axis('off')

print('Class label: ', labels[y_train[id][0]])

## 正解ラベルをone-hotベクトルに変換
`y_train`や`y_test`は、そのアドレスに対応する画像の正解ラベルが0から9までの整数値で記録されています。   
一方、1枚の入力画像に対してCNNが最終層で出力するのは、その画像が0から9までの各クラスである尤もらしさ（確率）ですから、それに対応するように、正解ラベルもone-hotベクトル（すなわち、正解のクラスだけが1、残りが0のベクトル）に変換しましょう。   

たとえば、正解のクラスが'frog'ならば、クラスラベルは`6`となっています。クラスは全部で10クラスですから、10次元のone-hotベクトルに変換します。すると、6番目だけが1で残りは0であるような`[0 0 0 0 0 0 1 0 0 0]`というベクトルに変換されます。   
これを、すべての画像（計50,000枚）について行うので、`y_train`は50,000x10の行列になります。

In [ ]:
# Convert class vectors to binary class matrices.
num_classes = 10 # 今回は10種類の物体を認識するので10クラス
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

`y_train`のサイズと、最初の10枚分の中身を見てみましょう。


In [ ]:
print('y_train shape:', y_train.shape)
print(y_train[:10])

## 画像の濃淡値を正規化

各画素の濃淡値は0～255までの整数値で記録されているので、255で割って0～1までの値に正規化します。

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Data Augmentation
機械学習では、訓練データの数は多ければ多いほど精度が高まるのが一般的です。   
そこで、限られた訓練データを仮想的に増やすため、データを様々に加工して新しいデータを生成する処理です。   
どのような画像が生成されているかを見たい方は、`DataAugmentation.ipynb`を参照してください。

In [ ]:
# 与えられた画像に対し、様々な処理を加えて新しい画像を生成するためジェネレータを作成
datagen = ImageDataGenerator(
        featurewise_center=False,  # 真理値．データセット全体で，入力の平均を0にします．
        samplewise_center=False,  # 真理値．各サンプルの平均を0にします．
        featurewise_std_normalization=False,  # 真理値．入力をデータセットの標準偏差で正規化します
        samplewise_std_normalization=False,  # 真理値．各入力をその標準偏差で正規化します．
        zca_whitening=False,  # 真理値．ZCA白色化を適用します．
        zca_epsilon=1e-06,  # ZCA白色化のイプシロン．デフォルトは1e-6
        rotation_range=20,  # 整数．画像をランダムに回転する回転範囲．
        width_shift_range=0.2, # 浮動小数点数（横幅に対する割合）．ランダムに水平シフトする範囲．
        height_shift_range=0.2, # 浮動小数点数（縦幅に対する割合）．ランダムに垂直シフトする範囲．
        shear_range=0.,  # 浮動小数点数．シアー強度（反時計回りのシアー角度）．
        zoom_range=0.,  # 浮動小数点数または[lower，upper]．ランダムにズームする範囲．浮動小数点数が与えられた場合，[lower, upper] = [1-zoom_range, 1+zoom_range]です．
        channel_shift_range=0.,  # 浮動小数点数．ランダムにチャンネルをシフトする範囲．
        fill_mode='nearest', # {"constant", "nearest", "reflect", "wrap"}のいずれか．デフォルトは 'nearest'です．指定されたモードに応じて，入力画像の境界周りを埋めます．
        cval=0.,  # 浮動小数点数または整数．fill_mode = "constant"のときに境界周辺で利用される値．
        horizontal_flip=True,  # 真理値．水平方向に入力をランダムに反転します．
        vertical_flip=False,  # 真理値．垂直方向に入力をランダムに反転します．
        rescale=None, # 画素値のリスケーリング係数．デフォルトはNone．Noneか0ならば，適用しない．それ以外であれば，(他の変換を行う前に) 与えられた値をデータに積算する．
        # set function that will be applied on each input
        preprocessing_function=None, # 各入力に適用される関数です．この関数は他の変更が行われる前に実行されます．この関数は3次元のNumpyテンソルを引数にとり，同じshapeのテンソルを出力するように定義する必要があります．
        data_format=None, # {"channels_first", "channels_last"}のどちらか．"channels_last"の場合，入力のshapeは(samples, height, width, channels)となり，"channels_first"の場合は(samples, channels, height, width)となります．デフォルトはKerasの設定ファイル~/.keras/keras.jsonのimage_data_formatの値です．一度も値を変更していなければ，"channels_last"になります．
        validation_split=0.0) # 浮動小数点数．検証のために予約しておく画像の割合（厳密には0から1の間）です．
#datagen.fit(x_train)

# 3. ネットワークを設計

上から順に画像データが通過していくと考えてください。   
なお、Dropoutとは、モデルが訓練データに過学習しないよう、ユニットのうちいくつかをランダムに無視するものです。

In [ ]:
model = Sequential()

# Convolution 1 フィルタ32枚、各フィルタのカーネルサイズ3x3 ストライドはデフォルト (1, 1)
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
# 'relu'で活性化
model.add(Activation('relu'))

# Convolution 2 フィルタ32枚、各フィルタのカーネルサイズ3x3 ストライドはデフォルト (1, 1)
model.add(Conv2D(32, (3, 3)))
# 'relu'で活性化
model.add(Activation('relu'))

# Max Pooling 1 (size: 2x2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# 25%のユニットをドロップアウト
model.add(Dropout(0.25))

# Convolution 3 フィルタ64枚、各フィルタのカーネルサイズ3x3 ストライドはデフォルト (1, 1)
model.add(Conv2D(64, (3, 3), padding='same'))
# 'relu'で活性化
model.add(Activation('relu'))

# Convolution フィルタ64枚、各フィルタのカーネルサイズ3x3 ストライドはデフォルト (1, 1)
model.add(Conv2D(64, (3, 3)))
# 'relu'で活性化
model.add(Activation('relu'))

# Max Pooling-2 (size: 2x2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# 25%のユニットをドロップアウト
model.add(Dropout(0.25))

# テンソルを一列のベクトルに平坦化
model.add(Flatten())

# Full Connection 1 # ユニット数512
model.add(Dense(512))
# 'relu'で活性化
model.add(Activation('relu'))

# 50%のユニットをドロップアウト
model.add(Dropout(0.5))

# Full Connection 2 # ユニット数はクラス数と同じ10
model.add(Dense(num_classes))
# 最後の活性化関数は出力を確率にするためsoftmaxを使用
model.add(Activation('softmax')) 

ネットワーク構造のサマリを出力してみましょう。

In [ ]:
model.summary()

# 4. 学習
## 最適化手法・損失関数・評価関数の設定
最適化手法を選択します。   
ここではよく使われる'Adam'と'RMSprop'のコードを書いておきます。   
損失関数は、今回は判別問題（Classification）なので`'categorical_crossentropy'`を指定します。   
もし回帰問題（Regression) ならば、`'mean_squared_error'`や`'mean_absolute_error'`を指定します。   
取り得る選択肢は[Kerasドキュメント](https://keras.io/ja/losses/)を参照してください。   
評価関数は`'accuracy'`としておきます。これは`'categorical_accuracy'`がデフォルト値です。詳しくは[Kerasドキュメント](https://keras.io/ja/metrics/)か[ソース](https://github.com/keras-team/keras/blob/c2e36f369b411ad1d0a40ac096fe35f73b9dffd3/keras/metrics.py)を参照してください。


In [ ]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              # RMSprop optimizer
              optimizer= keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
              # Adam optimizer
              # optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

## 学習パラメータ設定

以下のパラメータを設定
*   バッチサイズ：誤差を逆伝搬する際に、サンプルひとつずつ行うのではなく、いくつかのサンプルの誤差をまとめて逆伝搬します。そのときの1まとまりのサンプル数がバッチサイズです。1, 32, 128, 256, 512などが使われます。バッチサイズが大きいほど、特異値の影響を受けにくくなります。
*   エポック数：深層学習では同じ訓練データを何度も使ってパラメタを更新します。ここで指定するのは最大繰り返し回数です


In [ ]:
batch_size = 32 # バッチサイズ
epochs = 10 # エポック数

## 学習

上で設計したネットワークに訓練データを与えてモデルを学習します。   
**時間がかかりますので覚悟してください（お手持ちのPCに不安がある方は、Google Colaboratoryをご利用ください）。**

In [ ]:
data_augmentation = True # Data Augmentationを行わない場合の精度を見たい方は、ここをFalseにしてください

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:


    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


# 5. モデルの保存と評価

モデルの学習には長い時間がかかりますので、せっかく学習したモデルは保存しておきましょう。   
Colaboratoryで実行している場合は、保存後にPCにダウンロードしておかないと、ランタイムを停止する際に削除されてしまうのでご注意ください。

In [ ]:
save_dir = os.path.join(os.getcwd(), 'saved_models') # モデルの保存先
model_name = 'keras_cifar10_trained_model' # モデルを保存する際のファイル名
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

評価用データにより正解率を評価します。   

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# 6. 入力データに対するクラスラベルの予測

## 複数の評価データを一気に評価する

学習したモデルを使って、画像のクラスを予測してみましょう。   

評価データ`x_test`の最初の100枚に対して、クラスラベルを予測します。   
その後、正解ラベルとの混同行列を出力してみましょう。   
私の手元の結果では、`cat`を`dog`に誤りがちでしたが、皆さんはどうでしたか？


In [ ]:
from sklearn.metrics import confusion_matrix

predict_classes = model.predict_classes(x_test[:100,], batch_size=32, verbose=0)
true_classes = np.argmax(y_test[:100],1)
print('Confusion matrix:\n', confusion_matrix(true_classes, predict_classes))
print('Predicted labels:', predict_classes)

## 1枚の画像に対する認識結果を確認する

順番に、どの画像がどのように判定されたか見てみましょう。   

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# idをいろいろ入れ替えて、正解と予測を見比べてみましょう。
id = 100
plt.imshow(x_test[id])
ans = np.argmax(y_test[id])
plt.axis('off')

target = x_test[id] # 単体の入力データを用意
predict_class = model.predict_classes( np.array([target]) )
print(predict_class)
print('Truth: ', labels[ans], '\tPredicted: ', labels[predict_class[0]])

# 発展的演習
## Datasets
100クラスの一般物体認識タスクとして、[CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html)があります。  
変更点はわずかなのでぜひ試してみてください。   

## ネットワーク構造
モデルの構造を変えたらどうなるか試してみましょう。


# 参考文献
CIFAR-10/CIFAR-100: [Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.](https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf)

